In [1]:
import math

import pandas as pd

from konfuzio_sdk.data import Project, Document
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## An attempt into LR

In [2]:
my_project = Project(id_=1435)

In [7]:
ids = my_project.documents

In [8]:
ids

[Document Gehalt.pdf (282290),
 Document Festlohn.pdf (282291),
 Document vermögenswirksame Leistungen.pdf (282292),
 Document betriebliche Altersvorsorge AG finanziert.pdf (282293),
 Document Weihnachtsgeld.pdf (282294),
 Document Stundenlohn.pdf (282295),
 Document Fahrtkostenzuschuss pauschal versteuert.pdf (282296),
 Document Betirebliche Altersvorsorge Mischfinanzierung.pdf (282297),
 Document Darlehen.pdf (282298),
 Document Dienstwagen mit Gehaltsverzicht.pdf (282299),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_1.pdf (282300),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_2.pdf (282301),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_4.pdf (282302),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_3.pdf (282303),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_5.pdf (282304),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_6.pdf (282305),
 Document Auswertungs

In [10]:
meaningful = ['Label: Gesamt-Brutto', 'Label: Steuerrechtliche Abzüge', 'Label: Netto-Verdienst']

In [12]:
data = []
targets = []

for t in ids:
    try:
        annots = t.annotations()
        cur_data = []

        for annot in annots:
            if str(annot.label) in meaningful:
                if annot.normalized is not None:
                    cur_data.append([annot.label, annot.normalized])
            if str(annot.label) == 'Label: Auszahlungsbetrag':
                targets.append(annot.normalized)
        data.append(cur_data)
    except ValueError:
        pass

In [13]:
data[5][1][1] = 513.31

In [14]:
data = [[[x[0], float(str(x[1]).replace(',', '.'))] for x in y] for y in data]

In [15]:
data[4] = [['Label: Gesamt-Brutto', 4591.0],
 ['Label: Steuerrechtliche Abzüge', 1466.03],
 ['Label: Netto-Verdienst', 2182.68]]

In [16]:
data_clean = [[x[1] for x in y] for y in data]

In [17]:
df_x = pd.DataFrame(data_clean, dtype=float)

In [18]:
df_x.head()

,0,1,2
0,3120.00,292.11,2189.07
1,3000.00,207.99,2177.76
2,2759.19,400.72,1793.53
3,540.00,0.00,423.81
4,4591.00,1466.03,2182.68


In [19]:
df_y = pd.DataFrame(targets, dtype=float)

In [20]:
X = df_x
y = df_y

clf = LinearRegression()
clf.fit(X, y)

LinearRegression()

In [21]:
scores = clf.score(X, y)

In [22]:
scores

0.9375595440745054

In [25]:
test = my_project.test_documents

In [26]:
test_data = []
test_targets = []

for t in test:
    annots = t.annotations()
    cur_data = []
    
    for annot in annots:
        if str(annot.label) in meaningful:
            if annot.normalized is not None:
                cur_data.append([annot.label, annot.normalized])
        if str(annot.label) == 'Label: Auszahlungsbetrag':
            test_targets.append(annot.normalized)
    test_data.append(cur_data)

In [27]:
test_data = [[[x[0], float(str(x[1]).replace(',', '.'))] for x in y] for y in test_data]

In [28]:
test_data_clean = [[x[1] for x in y] for y in test_data]

In [29]:
test_data_clean

[[4355.4, 565.49, 3319.53],
 [4638.6, 589.73, 3569.5],
 [4675.78, 600.63, 3591.76]]

In [30]:
df_test = pd.DataFrame(test_data_clean, dtype=float)

In [31]:
pred = clf.predict(df_test)

In [32]:
pred

array([[2913.37823797],
       [3117.92172298],
       [3136.86300305]])

In [33]:
df_golden_standard = [[3482.08], [3462.82], [3576.53]]

In [34]:
print(mean_squared_error(df_golden_standard, pred))
print(math.sqrt(mean_squared_error(df_golden_standard, pred)))
print(mean_absolute_error(df_golden_standard, pred))

211894.5279449306
460.32002774692586
451.0890119998359


## An attempt into NER

In [3]:
import spacy
from spacy.tokens import DocBin

2022-07-25 13:08:04.323089: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-25 13:08:04.323111: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Training data preparation

In [98]:
ids[0].id_

282290

In [99]:
def prepare_data(text):
    text_id = text.id_
    
    with open('data_1435/documents/{}/document.txt'.format(text_id), 'r') as f:
        raw = f.read()
    
    result = []
    
    for annot in text.annotations():
        label = str(annot.label)[7:]
        if annot.bboxes.__len__ == 1:
            result.append((annot.bboxes[0].start_offset, annot.bboxes[0].end_offset, label))
        else:
            for el in annot.bboxes:
                result.append((el['start_offset'], el['end_offset'], label))
    
    return (raw, result)

In [101]:
training_data = []

for ix in ids:
    try:
        training_data.append(prepare_data(ix))
    except FileNotFoundError:
        pass

In [102]:
len(training_data)

25

In [103]:
nlp = spacy.blank("de")

db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")

In [4]:
nlp = spacy.load('output/model-best')

In [5]:
test_docs = my_project.test_documents

In [6]:
test_docs

[Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_18.pdf (282317),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_17.pdf (282318),
 Document Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_19.pdf (282319)]

In [47]:
def process_tests(text):
    text_id = text.id_
    
    with open('data_1435/documents/{}/document.txt'.format(text_id), 'r') as f:
        raw = f.read()
    
    test_line = raw.split('\n')[-9]
    
    doc = nlp(test_line)
    
    ents_in_line = []
    
    for ent in doc.ents:
        ents_in_line.append((ent.text, ent.label_))
    
    doc = nlp(raw)
    
    ents_in_text = []
    
    for ent in doc.ents:
        ents_in_text.append((ent.text, ent.label_))
    
    return ents_in_line, ents_in_text

In [48]:
test_1_ents_line, test_1_ents_text = process_tests(test_docs[0])
test_2_ents_line, test_2_ents_text = process_tests(test_docs[1])
test_3_ents_line, test_3_ents_text = process_tests(test_docs[2])

In [49]:
test_1_ents_line

[('DE33 7607 0024 0012 3456 78', 'Bank inkl. IBAN'),
 ('3.576,53', 'Auszahlungsbetrag')]

In [50]:
test_1_ents_text

[('21.12.2017', 'Austellungsdatum'),
 ('01111', 'Personalausweis'),
 ('3', 'Steuerklasse'),
 ('Karla-Muster', 'Vorname'),
 ('Klar', 'Nachname'),
 ('1200', 'Lohnart'),
 ('Überstundenzuschlag, 25%', 'Bezeichnung'),
 ('2,80', 'Menge'),
 ('24,40', 'Faktor'),
 ('17,08', 'Betrag'),
 ('1300', 'Lohnart'),
 ('Überstd.grundverg.+ FLA (25%)', 'Bezeichnung'),
 ('2,80', 'Menge'),
 ('24,40', 'Faktor'),
 ('68,32', 'Betrag'),
 ('2000', 'Lohnart'),
 ('Gehalt', 'Bezeichnung'),
 ('4.230,00', 'Betrag'),
 ('3100', 'Lohnart'),
 ('AG-Anteil VWL,1fd', 'Bezeichnung'),
 ('40,00', 'Betrag'),
 ('4.355,40', 'Gesamt-Brutto'),
 ('4.35540', 'Sozialversicherung'),
 ('51800', 'Sozialversicherung'),
 ('2814', 'Sozialversicherung'),
 ('1935', 'Sozialversicherung'),
 ('565,49', 'Steuerrechtliche Abzüge'),
 ('3.319,53', 'Netto-Verdienst'),
 ('57.83558', 'Steuer-Brutto'),
 ('9840', 'Lohnart'),
 ('9852', 'Lohnart'),
 ('9858', 'Lohnart'),
 ('Deutsche Bank PGK  Nürnbe', 'Bank inkl. IBAN'),
 ('DE33 7607 0024 0012 3456 78', 'Ban

In [10]:
with open('test_whole_text_1.txt', 'r') as f:
    test_data = f.read()

In [12]:
doc = nlp(test_data)

for ent in doc.ents:
    print((ent.text, ent.label_))

('21.12.2017', 'Austellungsdatum')
('01111', 'Personalausweis')
('3', 'Steuerklasse')
('Karla-Muster', 'Vorname')
('Klar', 'Nachname')
('DE33 7607 0024 0012 3456 78', 'Bank inkl. IBAN')
('3.576,53', 'Auszahlungsbetrag')
('1200', 'Lohnart')
('Überstundenzuschlag, 25%', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('17,08', 'Betrag')
('1300', 'Lohnart')
('Überstd.grundverg.+ FLA (25%)', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('68,32', 'Betrag')
('2000', 'Lohnart')
('Gehalt', 'Bezeichnung')
('4.230,00', 'Betrag')
('3100', 'Lohnart')
('AG-Anteil VWL,1fd', 'Bezeichnung')
('40,00', 'Betrag')
('4.355,40', 'Gesamt-Brutto')
('4.35540', 'Sozialversicherung')
('51800', 'Sozialversicherung')
('2814', 'Sozialversicherung')
('1935', 'Sozialversicherung')
('565,49', 'Steuerrechtliche Abzüge')
('3.319,53', 'Netto-Verdienst')
('57.83558', 'Steuer-Brutto')
('9840', 'Lohnart')
('9852', 'Lohnart')
('9858', 'Lohnart')
('Deutsche Bank PGK  Nürnbe', 'Bank inkl. IBAN')


In [13]:
with open('test_whole_text_2.txt', 'r') as f:
    test_data = f.read()

In [14]:
doc = nlp(test_data)

for ent in doc.ents:
    print((ent.text, ent.label_))

('21.12.2017', 'Austellungsdatum')
('01111', 'Personalausweis')
('3', 'Steuerklasse')
('Karla-Muster', 'Vorname')
('Klar', 'Nachname')
('Deutsche Bank PGK  Nürnbe', 'Bank inkl. IBAN')
('DE33 7607 0024 0012 3456 78', 'Bank inkl. IBAN')
('3.576,53', 'Auszahlungsbetrag')
('1200', 'Lohnart')
('Überstundenzuschlag, 25%', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('17,08', 'Betrag')
('1300', 'Lohnart')
('Überstd.grundverg.+ FLA (25%)', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('68,32', 'Betrag')
('2000', 'Lohnart')
('Gehalt', 'Bezeichnung')
('4.230,00', 'Betrag')
('3100', 'Lohnart')
('AG-Anteil VWL,1fd', 'Bezeichnung')
('40,00', 'Betrag')
('4.355,40', 'Gesamt-Brutto')
('4.35540', 'Sozialversicherung')
('51800', 'Sozialversicherung')
('2814', 'Sozialversicherung')
('1935', 'Sozialversicherung')
('565,49', 'Steuerrechtliche Abzüge')
('3.319,53', 'Netto-Verdienst')
('57.83558', 'Steuer-Brutto')
('9840', 'Lohnart')
('9852', 'Lohnart')
('9858', 'Lohnart')


In [17]:
with open('test_float_change_1.txt', 'r') as f:
    test_data = f.read()

In [18]:
doc = nlp(test_data)

for ent in doc.ents:
    print((ent.text, ent.label_))

('21.12.2017', 'Austellungsdatum')
('01111', 'Personalausweis')
('3', 'Steuerklasse')
('Karla-Muster', 'Vorname')
('Klar', 'Nachname')
('1200', 'Lohnart')
('Überstundenzuschlag, 25%', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('17,08', 'Betrag')
('1300', 'Lohnart')
('Überstd.grundverg.+ FLA (25%)', 'Bezeichnung')
('2,80', 'Menge')
('24,40', 'Faktor')
('68,32', 'Betrag')
('2000', 'Lohnart')
('Gehalt', 'Bezeichnung')
('4.230,00', 'Betrag')
('3100', 'Lohnart')
('AG-Anteil VWL,1fd', 'Bezeichnung')
('40,00', 'Betrag')
('4.355,40', 'Gesamt-Brutto')
('4.35540', 'Sozialversicherung')
('51800', 'Sozialversicherung')
('2814', 'Sozialversicherung')
('1935', 'Sozialversicherung')
('565,49', 'Steuerrechtliche Abzüge')
('3.319,53', 'Netto-Verdienst')
('57.83558', 'Steuer-Brutto')
('9840', 'Lohnart')
('9852', 'Lohnart')
('9858', 'Lohnart')
('Deutsche Bank PGK  Nürnbe', 'Bank inkl. IBAN')
('DE33 7607 0024 0012 3456 78', 'Bank inkl. IBAN')
('4.576,53', 'Auszahlungsbetrag')
